In [11]:
# login to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-2.amazonaws.com
# login to your private ECR
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 553020858742.dkr.ecr.us-east-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [12]:

! pygmentize -l docker Dockerfile.training

# Use Sagemaker PyTorch container as base image
# https://github.com/aws/sagemaker-pytorch-container/blob/master/docker/1.5.0/py3/Dockerfile.gpu
FROM 763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-training:1.5.0-gpu-py36-cu101-ubuntu16.04
LABEL author="vadimd@amazon.com"


############# Installing MMDetection from source ############

WORKDIR /opt/ml/code
RUN pip install --upgrade --force-reinstall  torch torchvision cython
RUN pip install mmcv-full==latest+torch1.5.0+cu101 -f https://openmmlab.oss-accelerate.aliyuncs.com/mmcv/dist/index.html

RUN git clone https://github.com/open-mmlab/mmdetection
RUN cd mmdetection/ && \
    pip install -e .

# to address https://github.com/pytorch/pytorch/issues/37377
ENV MKL_THREADING_LAYER GNU

############# Configuring Sagemaker ##############
COPY container_training /opt/ml/code

ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
ENV SAGEMAKER_PROGRAM mmdetection_train.py

WORKDIR /opt/ml/code/mmdetection

# Starts PyTorch distributed framework
E

In [1]:
! ./build_and_push.sh mmdetection-training latest Dockerfile.training

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  553.1MB
Step 1/13 : FROM 763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-training:1.5.0-gpu-py36-cu101-ubuntu16.04
 ---> 47cd15520b75
Step 2/13 : LABEL author="vadimd@amazon.com"
 ---> Using cache
 ---> 922dae94eea7
Step 3/13 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 1e829cda7967
Step 4/13 : RUN pip install --upgrade --force-reinstall  torch torchvision cython
 ---> Using cache
 ---> de5e1c07154c
Step 5/13 : RUN pip install mmcv-full==latest+torch1.5.0+cu101 -f https://openmmlab.oss-accelerate.aliyuncs.com/mmcv/dist/index.html
 ---> Using cache
 ---> 73fa494e5c7a
Step 6/13 : RUN git clone https://github.com/open-mmlab/mmdetecti

In [2]:

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:

import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = "us-east-2"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
prefix_input = 'mmdetection-input'
prefix_output = 'mmdetection-ouput'

In [4]:
container = "mmdetection-training" # your container name
tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container, tag)

In [5]:
hyperparameters = {
    "config-file" : "configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py",
    "dataset" : "coco"
    
}

In [6]:
metrics = []

## Local Training in SM Container

In [ ]:
est = sagemaker.estimator.Estimator(image,
                                          role=role,
                                          train_instance_count=2,
                                          train_instance_type='ml.p3.16xlarge',
#                                           train_instance_type="local_gpu", # use local_gpu for quick troubleshooting
                                          train_volume_size=100,
                                          output_path="s3://{}/{}".format(sess.default_bucket(), prefix_output),
                                          metric_definitions = metrics,
                                          hyperparameters = hyperparameters, 
#                                           sagemaker_session=sagemaker.LocalSession()
                                          sagemaker_session=sess
)

est.fit({"training" : "s3://coco2017-2a27f/coco"})

2020-07-25 22:32:10 Starting - Starting the training job...
2020-07-25 22:32:12 Starting - Launching requested ML instances.........
2020-07-25 22:33:48 Starting - Preparing the instances for training.........
2020-07-25 22:35:28 Downloading - Downloading input data...................................................................................................
2020-07-25 22:52:16 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-07-25 22:55:21,651 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-07-25 22:55:21,652 sagemaker-containers INFO     Failed to parse hyperparameter config-file value configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py to Json.
Returning the value itself
2020-07-25 22:55:21,653 sagemaker-containers INFO     Failed to parse hyperparameter dataset value coco to Json.
Returning the value itself
2020-